**As we have to deal with a multimodal classification task, DataLoader from `torchtext.data` cannot be used. This result in a prohibiting computation time if we compute the features on the fly.** \
**Furthemore, the BERT base model is a huge model and already very accurate so we will not fine-tune it. We want to use it as a feature extactor only.** \
<font color='red'> **==> We will precompute the textual features and save them in `torch.tensors`.** </font>

# Preliminaries

## Install and import libraries 

In [1]:
%%capture
!pip install transformers
import torch
from transformers import BertTokenizer, BertModel

## Set computation engine

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


## Connect to drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Load text data and compute features

In [4]:
source_folder = '/content/drive/MyDrive/data_rakuten' #source folder of csv files

In [5]:
#Tokenizer
tokenizer  = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-v2')

#BERT
encoder = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-v2')
for param in encoder.parameters():  #Freeze the weights of BERT because we want to use it only for sentence embedding
    param.requires_grad = False

encoder.eval()
encoder.cuda()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32768, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [8]:
#Load data
import pandas as pd
import os
import ast

XTrain = pd.read_csv(os.path.join(source_folder,'X_train_12tkObq.csv'), index_col=0)
XTest = pd.read_csv(os.path.join(source_folder,'X_test_gDTIJPh.csv'), index_col=0)

YTrain = pd.read_csv(os.path.join(source_folder,'y_train_Q9n2dCu.csv'), index_col=0)
YTrain['color_tags'] = YTrain['color_tags'].apply(lambda x: ast.literal_eval(x)) #to change str to list labels

In [10]:
def compute_feature(text):
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor  = torch.tensor([indexed_tokens]).to(device)
    tokens_tensor=tokens_tensor[:,:512] #to prevent tokens sequence longer than 512 tokens
    text_features  = encoder.forward(input_ids=tokens_tensor,return_dict=True)
    pooler_output  = text_features['pooler_output'].squeeze(0)

    return  pooler_output


In [11]:
Xtrain_item_caption=torch.zeros([768,len(XTrain)])   # a big guy :p

for idx,item_caption in enumerate(XTrain["item_caption"]):
    Xtrain_item_caption[:,idx] = compute_feature("item_caption")
    if idx%1000==0:
      print(idx)

torch.save(Xtrain_item_caption,'/content/drive/MyDrive/data_rakuten/Xtrain_item_caption.pt')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [12]:
Xtrain_item_name=torch.zeros([768,len(XTrain)])   # another big guy :p

for idx,item_name in enumerate(XTrain["item_name"]):
    Xtrain_item_name[:,idx] = compute_feature("item_name")
    if idx%1000==0:
      print(idx)

torch.save(Xtrain_item_name,'/content/drive/MyDrive/data_rakuten/Xtrain_item_name.pt')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [13]:
Xtest_item_caption=torch.zeros([768,len(XTest)])   # another big guy :p

for idx,item_caption in enumerate(XTest["item_caption"]):
    Xtest_item_caption[:,idx] = compute_feature("item_caption")
    if idx%1000==0:
      print(idx)

torch.save(Xtest_item_caption,'/content/drive/MyDrive/data_rakuten/Xtest_item_caption.pt')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000


In [14]:
Xtest_item_name=torch.zeros([768,len(XTest)])   # another big guy :p

for idx,item_name in enumerate(XTest["item_name"]):
    Xtest_item_name[:,idx] = compute_feature("item_name")
    if idx%1000==0:
      print(idx)

torch.save(Xtest_item_name,'/content/drive/MyDrive/data_rakuten/Xtest_item_name.pt')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000


In [15]:
dico_labels={  "Beige": 0,  "Black": 1,  "Blue": 2,  "Brown": 3,  "Burgundy": 4,  "Gold": 5,  "Green": 6,  
  "Grey": 7,    "Khaki": 8,  "Multiple Colors": 9,  "Navy": 10,  "Orange": 11,
  "Pink": 12,"Purple": 13,"Red": 14,"Silver": 15, "Transparent": 16,"White": 17,"Yellow": 18  }

In [16]:
Ytrain_label=torch.zeros([19,len(YTrain)])   # another big guy :p

for idx,str_labels in enumerate(YTrain['color_tags']):
    int_labels=[dico_labels[color] for color in str_labels]
    tensor_labels=torch.zeros(19)
    for label in int_labels:
        Ytrain_label[label,idx]=1

    if idx%1000==0:
      print(idx)

torch.save(Ytrain_label,'/content/drive/MyDrive/data_rakuten/Ytrain_label.pt')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
